# Notebook de mejora

Este notebook es un ejemplo cómo se puede mejorar los hiperparámetros de un algoritmo, y medir la importancia de atributos https://www.kaggle.com/c/house-prices-advanced-regression-techniques/ de la práctica 3 de Inteligencia de Negocio. 

Incluyo el código del notebook anterior, no se explicarán.

Es un ejemplo que es claramente mejorable, se deja a el/la estudiante el mejorarlo para obtener mejores resultados.

In [49]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score


In [50]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv", na_values="NaN") # Definimos na_values para identificar bien los valores perdidos


if 'Id' in train:
    train.drop('Id', axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

test = pd.read_csv("test.csv", na_values="NaN")
test_ids = test.Id
test = test.drop('Id', axis=1)

input_all = pd.concat([train.drop('SalePrice', axis=1), test])

col_cat = list(input_all.select_dtypes(exclude=np.number).columns)

from sklearn.impute import SimpleImputer

# Valores categóricos por el más frecuente
imputer_cat = SimpleImputer(strategy="most_frequent")
imputer_cat.fit(input_all[col_cat])
train[col_cat] = imputer_cat.transform(train[col_cat])
test[col_cat] = imputer_cat.transform(test[col_cat])

# Valores numéricos por KNN
col_num = list(train.select_dtypes(include=np.number).columns)
col_num.remove('SalePrice')

# Configuración del imputador KNN
knn_imputer = KNNImputer(n_neighbors=5)  # Puedes ajustar el número de vecinos según tus necesidades

# Imputación para el conjunto de entrenamiento
train[col_num] = knn_imputer.fit_transform(train[col_num])

# Imputación para el conjunto de prueba
test[col_num] = knn_imputer.transform(test[col_num])

from sklearn.preprocessing import LabelEncoder
labelers = {}
test_l = test.copy()
train_l = train.copy()


y_train = train_l.SalePrice
X_train = train_l.drop('SalePrice', axis=1)

if 'Id' in test_l:
    test_l.drop('Id', axis=1, inplace=True)

X_test = test_l

In [51]:
from sklearn.compose import make_column_selector

cat_selector = make_column_selector(dtype_include=object)
num_selector = make_column_selector(dtype_include=np.number)

In [52]:
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler

cat_linear_processor = OneHotEncoder(handle_unknown="ignore")
num_linear_processor = StandardScaler()

linear_preprocessor = make_column_transformer(
    (num_linear_processor, num_selector), (cat_linear_processor, cat_selector)
)
linear_preprocessor

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78A3E1DC0>),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78F11A670>)])

In [53]:
cat_tree_processor = OrdinalEncoder(
    handle_unknown="use_encoded_value",
    unknown_value=-1,
    encoded_missing_value=-2,
)

num_tree_processor = SimpleImputer(strategy="mean", add_indicator=True)

tree_preprocessor = make_column_transformer(
    (num_tree_processor, num_selector), (cat_tree_processor, cat_selector)
)
tree_preprocessor

ColumnTransformer(transformers=[('simpleimputer',
                                 SimpleImputer(add_indicator=True),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78A3E1DC0>),
                                ('ordinalencoder',
                                 OrdinalEncoder(encoded_missing_value=-2,
                                                handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78F11A670>)])

In [54]:
from sklearn.ensemble import RandomForestRegressor

rf_pipeline = make_pipeline(tree_preprocessor, RandomForestRegressor(criterion='squared_error', max_depth=20, random_state=42))
rf_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(add_indicator=True),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78A3E1DC0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-2,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78F11A670>)])),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=20, random_state=42))])

In [55]:
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# KNN pipeline
knn_pipeline = make_pipeline(tree_preprocessor, KNeighborsRegressor())

knn_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(add_indicator=True),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78A3E1DC0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-2,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78F11A670>)])),
                ('kneighborsregressor', KNeighborsRegressor())])

In [56]:
# GradientBoostingRegressor pipeline
gradient_boosting_pipeline = make_pipeline(
    tree_preprocessor, GradientBoostingRegressor(random_state=42)
)

gradient_boosting_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(add_indicator=True),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78A3E1DC0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-2,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78F11A670>)])),
                ('gradientboostingregressor',
                 GradientBoostingRegressor(random_state=42))])

In [57]:
# XGBoost pipeline
xgboost_pipeline = make_pipeline(
    tree_preprocessor, XGBRegressor(objective='reg:squarederror', random_state=42)
)

xgboost_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(add_indicator=True),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78A3E1DC0>),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(encoded_missing_value=-2,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transfor...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [58]:
from sklearn.linear_model import LassoCV

lasso_pipeline = make_pipeline(linear_preprocessor, LassoCV())
lasso_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78A3E1DC0>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78F11A670>)])),
                ('lassocv', LassoCV())])

In [59]:
# Collect predictions and errors from each base estimator
def collect_predictions_errors(estimator, X, y):
    predictions = estimator.predict(X)
    errors = y - predictions
    return predictions.reshape(-1, 1), errors.reshape(-1, 1)


In [60]:
# Define the base models
base_models = [
    ('random_forest', rf_pipeline, collect_predictions_errors),
    ('knn', knn_pipeline, collect_predictions_errors),
    ('gradient_boosting', gradient_boosting_pipeline, collect_predictions_errors),
    ('xgboost', xgboost_pipeline, collect_predictions_errors),
    ('lasso', lasso_pipeline)
]

stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=Ridge())
stacking_regressor

StackingRegressor(estimators=[('random_forest',
                               Pipeline(steps=[('columntransformer',
                                                ColumnTransformer(transformers=[('simpleimputer',
                                                                                 SimpleImputer(add_indicator=True),
                                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78A3E1DC0>),
                                                                                ('ordinalencoder',
                                                                                 OrdinalEncoder(encoded_missing_value=-2,
                                                                                                handle_unknown='use_encoded_value',
                                                                                                unkn...
                               Pipeline(steps=[('columntransformer',
                                                ColumnTransformer(transformers=[('standardscaler',
                                                                                 StandardScaler(),
                                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78A3E1DC0>),
                                                                                ('onehotencoder',
                                                                                 OneHotEncoder(handle_unknown='ignore'),
                                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78F11A670>)])),
                                               ('lassocv', LassoCV())]))],
                  final_estimator=Ridge())

In [61]:
from sklearn.model_selection import KFold

# Define the cross-validation strategy
cv = KFold(n_splits=10, shuffle=True, random_state=124345)

# Calculate cross-validated negative mean squared log error
values = cross_val_score(stacking_regressor, X_train, y_train, scoring='neg_mean_squared_log_error', cv=cv, error_score='raise')

# Print the results
print(values)
print(values.mean())

C:\Users\germa\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4431800720.807434, tolerance: 715421553.2221366
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\germa\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3794941250.61322, tolerance: 676922175.5669698
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\germa\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5740523610.473572, tolerance: 719876660.4098223
  model = cd_fast.sparse_enet_coordinate_descent(


[-0.00826415 -0.0192129  -0.01282036 -0.01345516 -0.01224757 -0.02541899
 -0.02449989 -0.01409799 -0.02033022 -0.01173722]
-0.016208444877600453


In [62]:
stacking_regressor.fit(X_train, y_train)

StackingRegressor(estimators=[('random_forest',
                               Pipeline(steps=[('columntransformer',
                                                ColumnTransformer(transformers=[('simpleimputer',
                                                                                 SimpleImputer(add_indicator=True),
                                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78A3E1DC0>),
                                                                                ('ordinalencoder',
                                                                                 OrdinalEncoder(encoded_missing_value=-2,
                                                                                                handle_unknown='use_encoded_value',
                                                                                                unkn...
                               Pipeline(steps=[('columntransformer',
                                                ColumnTransformer(transformers=[('standardscaler',
                                                                                 StandardScaler(),
                                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78A3E1DC0>),
                                                                                ('onehotencoder',
                                                                                 OneHotEncoder(handle_unknown='ignore'),
                                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000001D78F11A670>)])),
                                               ('lassocv', LassoCV())]))],
                  final_estimator=Ridge())

In [63]:
pred = stacking_regressor.predict(X_test)

In [64]:
salida = pd.DataFrame({'Id': test_ids, 'SalePrice': pred})
salida.to_csv("Prueba7.csv", index=False)